# SOAP Using Session Key and Writing to Pandas DataFrame

In [ ]:
import datetime
import zeep
from zeep import Client
from zeep.transports import Transport
from requests import Session
from requests.auth import HTTPBasicAuth
import pandas as pd

username = ""
password = ""
wsdl = ""

session = Session()
session.auth = HTTPBasicAuth(username, password)
client = Client(wsdl, transport=Transport(session=session))

SessionKey = client.service.SessionStart(Input=SessionStartInputHeader)
key = SessionKey.Result.DataRows.DataRow[0].SessionKey

try:

    servicesessiondata = {
        'Key': key
    }

    servicesessioninputheader = {"Parameters": servicesessiondata}
    service_key = client.service.ServiceSessionStart(Input=servicesessioninputheader)
    service_key = service_key.Result.DataRows.DataRow[0].ServiceSessionKey

    request_data = {
        "Parameters": {
            
        },
        "Header": {'ServiceSessionKey': service_key}}

    response = client.service.FunctionName(Input=request_data)

    # manipulating response
    result = zeep.helpers.serialize_object(response)

    df = pd.DataFrame(result['Result']['DataRows']['DataRow'])

    df['LoadDate'] = datetime.now()
    df['LoadSource'] = "Python Script"

except Exception as e:

    print(e)

finally:

    # for terminating the session run:
    terminate_key = {"Header": key}
    client.service.SessionEnd(Input=terminate_key)